In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ThemeParkBot

This ThemeParkBot is designed to help the management and creative decisions about the theme park, suggesting which type of attractions are most popular, which attractions are a net loss and should be replaced and so on...

### Install SDK

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.6 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [6]:
%load_ext sql
%sql sqlite:///ThemePark.db

In [7]:
%%sql
-- Create the 'Land' table
CREATE TABLE IF NOT EXISTS Land (
  	LandId INTEGER PRIMARY KEY AUTOINCREMENT,
  	LandName VARCHAR(255) NOT NULL,
  	LandArea DECIMAL(10, 2) NOT NULL
  );

-- Create the 'AttractionType' table
CREATE TABLE IF NOT EXISTS AttractionType (
  	AttractionTypeId INTEGER PRIMARY KEY AUTOINCREMENT,
  	AttractionType VARCHAR(255) NOT NULL  	
  );

-- Create the 'Attraction' table
CREATE TABLE IF NOT EXISTS Attraction (
  	AttractionId INTEGER PRIMARY KEY AUTOINCREMENT,
  	Attraction VARCHAR(255) NOT NULL,
  	AttractionTypeId INTEGER NOT NULL,
  	LandId INTEGER NOT NULL,
  	FOREIGN KEY (AttractionTypeId) REFERENCES AttractionType (AttractionTypeId),
    FOREIGN KEY (LandId) REFERENCES Land (LandId)
  );

-- Create the 'AttractionPerYear' table
CREATE TABLE IF NOT EXISTS AttractionPerYear (
  	AttractionPerYearId INTEGER PRIMARY KEY AUTOINCREMENT,
    AttractionId INTEGER,
  	Year INTEGER NOT NULL,
    Revenue DECIMAL(12, 2) NOT NULL,
  	Costs DECIMAL(12, 2) NOT NULL,
    Guests INTEGER NOT NULL DEFAULT 0,
  	IsOperational BOOLEAN NOT NULL DEFAULT 1,
  	FOREIGN KEY (AttractionId) REFERENCES Attraction (AttractionId)
  );

INSERT INTO Land (LandName, LandArea)
SELECT * FROM (
  SELECT 'Adventure' AS LandName, 12.50 AS LandArea
  UNION ALL
  SELECT 'Fantasy', 9.75
  UNION ALL
  SELECT 'Steampunk', 14.30
) AS new_lands
WHERE NOT EXISTS (
  SELECT 1 FROM Land WHERE LandName = new_lands.LandName
);

-- Insert into AttractionType
INSERT INTO AttractionType (AttractionType)
SELECT * FROM (
  SELECT 'Roller Coaster' AS AttractionType
  UNION ALL
  SELECT 'Water Ride'
  UNION ALL
  SELECT 'Dark Ride'
) AS new_attraction_types
WHERE NOT EXISTS (
  SELECT 1 FROM AttractionType WHERE AttractionType = 'Roller Coaster'
);

-- Insert into Attraction
INSERT INTO Attraction (Attraction, AttractionTypeId, LandId)
SELECT * FROM (
  SELECT 'Ghost House' AS Attraction, 3 AS AttractionTypeId, 2 AS LandId
  UNION ALL
  SELECT 'Splash Zone', 2, 1
  UNION ALL
  SELECT 'Space Voyage', 1, 2
) AS new_attractions
WHERE NOT EXISTS (
  SELECT 1 FROM Attraction WHERE Attraction = 'Ghost House'
);

-- Insert into AttractionPerYear
INSERT INTO AttractionPerYear (AttractionId, Year, Revenue, Costs, Guests, IsOperational)
SELECT * FROM (
  SELECT 1 AS AttractionId, 2024 AS Year, 1200000.00 AS Revenue, 450000.00 AS Costs, 250000 AS Guests, 1 AS IsOperational
  UNION ALL
  SELECT 2, 2024, 9800.00, 300000.00, 1800, 1
  UNION ALL
  SELECT 3, 2024, 750000.00, 270000.00, 120000, 1
  UNION ALL
  SELECT 1, 2023, 120000.00, 450000.00, 250000, 1
  UNION ALL
  SELECT 2, 2023, 98000.00, 300000.00, 18000, 1
  UNION ALL
  SELECT 3, 2023, 75000.00, 270000.00, 12000, 1
  UNION ALL
  SELECT 1, 2022, 12000.00, 450000.00, 25000, 1
  UNION ALL
  SELECT 2, 2022, 980000.00, 300000.00, 180000, 1
  UNION ALL
  SELECT 3, 2022, 7500.00, 270000.00, 1200, 1
) AS new_attraction_per_year
WHERE NOT EXISTS (
  SELECT 1 FROM AttractionPerYear WHERE AttractionId = 1 AND Year = 2024
);



 * sqlite:///ThemePark.db
Done.
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.
9 rows affected.


[]

# Database functions

In [8]:
import sqlite3

db_file = "ThemePark.db"
db_conn = sqlite3.connect(db_file)

Function to list all the tables

In [9]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.

    cursor = db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]

list_tables()

['Land',
 'sqlite_sequence',
 'AttractionType',
 'Attraction',
 'AttractionPerYear']

Function to describe table schema

In [10]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.
    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    cursor = db_conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]

describe_table("Land")

[('LandId', 'INTEGER'),
 ('LandName', 'VARCHAR(255)'),
 ('LandArea', 'DECIMAL(10, 2)')]

Function to run a query

In [11]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    cursor = db_conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

execute_query("select * from Land")

[(1, 'Adventure', 12.5), (2, 'Fantasy', 9.75), (3, 'Steampunk', 14.3)]

# Implement functions

In [12]:
# These are the Python functions defined above.
available_functions = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a theme park. You will take the users questions and - if needed - turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=available_functions,
    ),
)

### Conversations

In [13]:
resp = chat.send_message("Which is the most popular attraction?")
print(f"\n{resp.text}")


The most popular attraction is Ghost House.



In [14]:
resp = chat.send_message("What has been the profit for the attraction Ghost House last year (2024)?")
print(f"\n{resp.text}")


The profit for Ghost House in 2024 was 750000.



In [15]:
resp = chat.send_message("What have been the profits for all the attractions last year (2024)? Order them in descending profit order this way: -OrderNumber AttractionName Profit. Nothing else")
print(f"\n{resp.text}")


Here are the profits for all the attractions in 2024, ordered by descending profit:

1 Ghost House 750000
2 Space Voyage 480000
3 Splash Zone -290200



In [16]:
resp = chat.send_message("I want to build a new attraction, which old attraction shuld I replace it with? Reason step by step")
print(f"\n{resp.text}")


Based on the analysis, Splash Zone had the lowest profit (-290200) and a guest count of 1800 in 2024. Therefore, Splash Zone should be replaced with a new attraction.



In [17]:
resp = chat.send_message("What have been the profits for all the attractions last year (2024)? Generate and run some code to plot this as a python seaborn chart")
print(f"\n{resp.text}")


Here's the data you requested, which you can use to create a Seaborn chart:

| Attraction    | Profit   |
| :------------ | :------- |
| Ghost House   | 750000   |
| Space Voyage  | 480000   |
| Splash Zone   | -290200  |

You can use this data to create a bar chart with Seaborn, where the x-axis represents the attraction name and the y-axis represents the profit. This will visually show the profit for each attraction in 2024.

